## Load the dataset

In [29]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
train = 'Dataset/OCT2017/train'
test = 'Dataset/OCT2017/test'
#rescale parameter equal to 1/255 to normalize these values
train_datagen = ImageDataGenerator(rescale=(1./255))
test_datagen = ImageDataGenerator(rescale=(1./255))
training_set = train_datagen.flow_from_directory(directory = train,target_size=(224,224),batch_size=128,class_mode = "categorical")
test_set = test_datagen.flow_from_directory(directory = test,target_size=(224,224),batch_size=128,class_mode = "categorical")

Found 83484 images belonging to 4 classes.
Found 968 images belonging to 4 classes.


In [19]:
image = cv2.imread('Dataset/OCT2017/train/CNV/CNV-81630-8.jpeg')
image.size 

761856

## Masking
#### Masking in image processing refers to the process of isolating or extracting a specific region or object of interest within an image by applying a binary mask. The mask is essentially a binary image of the same dimensions as the original image, where each pixel is marked as either part of the region of interest (ROI) or not. A pixel in the mask is typically set to 1 if it belongs to the ROI and 0 if it does not.

In [20]:
# image = cv2.imread("Dataset/OCT2017/train/DRUSEN/DRUSEN-228939-51.jpeg", cv2.IMREAD_GRAYSCALE)

def enhance_contrast(image):
    enhanced_image = cv2.equalizeHist(image)
    return enhanced_image

# cv2.imwrite("enhanced_image.jpg", enhanced_image)
# cv2.imshow('enhance', enhanced_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [21]:
def create_binary_image(image):
    _, binary_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)
    inverted_binary_image = cv2.bitwise_not(binary_image)
    return inverted_binary_image

In [22]:
def perform_edge_detection(image):
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    edges = cv2.Canny(blurred, threshold1=30, threshold2=70)
    return edges
# cv2.imshow('Original Image', image)
# cv2.imshow('Edges', edges)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [23]:
def apply_morphological_operations(edges):
    kernel = np.ones((3, 3), np.uint8)
    for _ in range(3):
        edges = cv2.dilate(edges, kernel, iterations=1)
    for _ in range(12):
        edges = cv2.erode(edges, kernel, iterations=1)
    return edges
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [24]:
def find_contours(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

In [25]:
def extract_largest_roi(image, contours):
    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        mask = np.zeros_like(image)
        cv2.drawContours(mask, [largest_contour], -1, (255), thickness=cv2.FILLED)
        roi = cv2.bitwise_and(image, image, mask=mask)
        return roi
    else:
        return None
 # -1 means draw all contours

In [11]:
def invert_roi(roi):
    inverted_roi = cv2.bitwise_not(roi)
    return inverted_roi
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [12]:
def extract_and_save_roi(inverted_roi, original_image_path, output_path):
    original_image = cv2.imread(original_image_path)
    mask = cv2.cvtColor(inverted_roi, cv2.COLOR_GRAY2BGR)  # Convert inverted_roi to a BGR mask
    extracted_roi = cv2.bitwise_and(original_image, mask)
    cv2.imwrite(output_path, extracted_roi)

In [13]:
def process_and_save_image(input_path, output_path):
    image = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
    enhanced_image = enhance_contrast(image)
    binary_image = create_binary_image(enhanced_image)
    edges = perform_edge_detection(binary_image)
    morphological_edges = apply_morphological_operations(edges)
    contours = find_contours(morphological_edges)
    roi = extract_largest_roi(morphological_edges, contours) 
    if roi is not None:
        inverted_roi = invert_roi(roi)
        extract_and_save_roi(inverted_roi, input_path, output_path)

In [14]:
def process_image_folder(input_folder, output_folder):
    for class_label in os.listdir(input_folder):
        class_input_folder = os.path.join(input_folder, class_label)
        class_output_folder = os.path.join(output_folder, class_label)
        os.makedirs(class_output_folder, exist_ok=True)
        for image_file in os.listdir(class_input_folder):
            input_path = os.path.join(class_input_folder, image_file)
            output_path = os.path.join(class_output_folder, image_file)
            process_and_save_image(input_path, output_path)

In [16]:
input_folder = 'Dataset/OCT2017/train'
output_folder = 'ProcessedImages'
process_image_folder(input_folder, output_folder)

KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np
original_image = cv2.imread('Dataset/OCT2017/train/DRUSEN/DRUSEN-228939-51.jpeg')
mask = cv2.imread('roi.jpg', cv2.IMREAD_GRAYSCALE).astype(np.uint8)
extracted_roi = cv2.bitwise_and(original_image, original_image, mask=mask)
difference_image = cv2.absdiff(original_image, extracted_roi)
cv2.imshow('ROI', extracted_roi)
cv2.imwrite("difference_image.jpg",difference_image)
cv2.imshow('Difference Image', difference_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [40]:
processed_train = 'ProcessedImages'
#rescale parameter equal to 1/255 to normalize these values
processed_train_datagen = ImageDataGenerator(rescale=(1./255))
training_set = processed_train_datagen.flow_from_directory(directory = processed_train,target_size=(224,224),batch_size=128,class_mode = "categorical")

Found 83154 images belonging to 4 classes.


In [41]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    x = Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(shortcut)
    
    x = tf.keras.layers.Add()([x, shortcut])
    x = ReLU()(x)
    return x

def resnet_2d(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    
    x = Conv2D(64, 7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=3, strides=2, padding='same')(x)
    
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 128, stride=2)
    x = residual_block(x, 128)
    x = residual_block(x, 256, stride=2)
    x = residual_block(x, 256)
    x = GlobalAveragePooling2D()(x)
    
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)
    return model

# Define your input shape and number of classes
input_shape = (224, 224, 3)  # Adjust input shape according to your data
num_classes = 4  # Change num_classes according to your task

# Create the ResNet-18 model
model = resnet_2d(input_shape, num_classes)

# Print the model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_15 (Conv2D)          (None, 112, 112, 64)         9472      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_13 (Ba  (None, 112, 112, 64)         256       ['conv2d_15[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_13 (ReLU)             (None, 112, 112, 64)         0         ['batch_normalization_13

 conv2d_24 (Conv2D)          (None, 28, 28, 128)          147584    ['re_lu_20[0][0]']            
                                                                                                  
 batch_normalization_21 (Ba  (None, 28, 28, 128)          512       ['conv2d_24[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 add_9 (Add)                 (None, 28, 28, 128)          0         ['batch_normalization_21[0][0]
                                                                    ',                            
                                                                     're_lu_19[0][0]']            
                                                                                                  
 re_lu_21 (ReLU)             (None, 28, 28, 128)          0         ['add_9[0][0]']               
          

In [50]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Choose the appropriate loss function
    metrics=['accuracy']  # Add more metrics as needed
)

In [ ]:
model.fit(
    x=training_set,  
    validation_data= test_set, 
    epochs=10,
)

Epoch 1/10
653/653 [==============================] - 7324s 11s/step - loss: 0.3265 - accuracy: 0.8858 - val_loss: 0.3402 - val_accuracy: 0.8585
Epoch 2/10
653/653 [==============================] - 33805s 52s/step - loss: 0.1949 - accuracy: 0.9335 - val_loss: 0.1402 - val_accuracy: 0.9556
Epoch 3/10
653/653 [==============================] - 12940s 20s/step - loss: 0.1607 - accuracy: 0.9450 - val_loss: 0.3049 - val_accuracy: 0.8771
Epoch 4/10
653/653 [==============================] - 7641s 12s/step - loss: 0.1409 - accuracy: 0.9521 - val_loss: 0.0718 - val_accuracy: 0.9742
Epoch 5/10
653/653 [==============================] - 8241s 13s/step - loss: 0.1264 - accuracy: 0.9561 - val_loss: 0.0782 - val_accuracy: 0.9824
Epoch 6/10
653/653 [==============================] - 8008s 12s/step - loss: 0.1151 - accuracy: 0.9602 - val_loss: 0.0164 - val_accuracy: 0.9969
Epoch 7/10
653/653 [==============================] - 9243s 14s/step - loss: 0.1043 - accuracy: 0.9632 - val_loss: 0.0512 - val_

In [51]:
image_path = r"Dataset\OCT2017\val\CNV\CNV-6294785-2.jpeg"
new_img = image.load_img(image_path, target_size=(224, 224))

In [52]:
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
img = img/255
prediction = model.predict(img)

1/1 [==============================] - 1s 959ms/step


In [53]:
prediction

array([[0.2576708 , 0.24667235, 0.24488547, 0.2507713 ]], dtype=float32)

In [54]:
predicted_class = np.argmax(prediction)

In [55]:
print("Class predicted " + str(predicted_class))

Class predicted 0


In [56]:
model.save("retinal_oct_model.h5")

C:\Users\Dhurika\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [57]:
import tensorflow as tf
model = tf.keras.models.load_model('retinal_oct_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('retinal_oct_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Dhurika\AppData\Local\Temp\tmpk4hjtijv\assets


INFO:tensorflow:Assets written to: C:\Users\Dhurika\AppData\Local\Temp\tmpk4hjtijv\assets
